In [1]:
import pandas as pd
import numpy as np
import random
import kagglehub

In [2]:
d2 = kagglehub.dataset_download("benroshan/online-food-delivery-preferencesbangalore-region")

print("Path to dataset files:", d2)

d5 = kagglehub.dataset_download("saurabhbadole/zomato-delivery-operations-analytics-dataset")

print("Path to dataset files:", d5)

100%|██████████| 24.0k/24.0k [00:00<00:00, 11.3MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/benroshan/online-food-delivery-preferencesbangalore-region/versions/3


100%|██████████| 1.43M/1.43M [00:00<00:00, 105MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/saurabhbadole/zomato-delivery-operations-analytics-dataset/versions/1


Dataset 5

In [3]:
df_5 = pd.read_csv("/root/.cache/kagglehub/datasets/saurabhbadole/zomato-delivery-operations-analytics-dataset/versions/1/Zomato Dataset.csv")

For column Time_Orderd, remove Nan, 0.x and 1 values:

In [ ]:
df_5 = df_5[~df_5['Time_Orderd'].apply(lambda x: isinstance(x, float) or '0.' in str(x) or str(x) == '1')]

For column Time_Order_picked, remove 0.x and 1 values:

In [ ]:
df_5 = df_5[~df_5['Time_Order_picked'].apply(lambda x: '0.' in str(x) or str(x) == '1' or '24:' in str(x))]

Cleaning Dataset D5

In [14]:
df_5_cleaned = df_5.dropna()

D5 is now cleaned!

Now create a new dataset by simulation: Generate 34326 values of date and time

In [7]:
num_rows = 34326
from datetime import datetime, timedelta
# Define the start and end dates
start_date = "2022-04-01"
end_date = "2024-03-31"
# Create a range of dates
date_range = pd.date_range(start=start_date, end=end_date)
# Sample random dates from the range
random_dates = np.random.choice(date_range, size=num_rows, replace=True)
# Create a DataFrame with the random dates
df_5_new = pd.DataFrame(random_dates, columns=['Date'])
# Add a random time component to each date
df_5_new['Date'] = df_5_new['Date'] + pd.to_timedelta(np.random.randint(0, 24 * 60 * 60, size=len(df_5_new)), unit='s')


Split above generated data into 2 new columns Order_Date and Time_Orderd:

In [8]:
# Assuming 'Date' column is already a datetime type

# Extract date and time components using vectorized operations
df_5_new['Order_Date'] = df_5_new['Date'].dt.strftime("%d-%m-%Y")
df_5_new['Time_Orderd'] = df_5_new['Date'].dt.strftime("%H:%M")

df_5_new = df_5_new.drop('Date', axis=1)


Drop existing column Date as it is not needed now:

In [ ]:
df_5_new = df_5_new.drop('Date', axis=1)

Generate new column Time_Order_picked based on Time_Orderd column:

In [9]:
def add_random_minutes(time_str):
    # Convert 'time_ordered' to a datetime object
    time_obj = pd.to_datetime(time_str, format='%H:%M')

    # Generate a random value between 5 and 30 (minutes)
    random_minutes = np.random.randint(5, 31)

    # Add the random minutes to the time
    new_time_obj = time_obj + pd.Timedelta(minutes=random_minutes)

    if new_time_obj.hour == 24:
        new_time_obj = new_time_obj.replace(hour=0, minute=new_time_obj.minute)

    # Return the new time as a string in HH:MM format
    return new_time_obj.strftime('%H:%M')
# Create a new column with the random added minutes
df_5_new['Time_Order_picked'] = df_5_new['Time_Orderd'].apply(add_random_minutes)

Generate new column Weather_conditions:

In [10]:
# List of possible weather conditions
weather_conditions = ['Windy','Sandstorms', 'Fog', 'Stormy', 'Cloudy', 'Rainy', 'Sunny']
weather_frequencies = [10, 5, 5, 15, 15, 30, 50]
# Randomly assign weather conditions to the 'weather' column
df_5_new['Weather_conditions'] = np.random.choice(weather_conditions, size=num_rows, p=np.array(weather_frequencies) / sum(weather_frequencies))

We have now generated a new dataset with required values!

Now we have to merge df_5 and df_5_new:

In [11]:
final_df_5 = pd.DataFrame()

df_5_columns = ['Order_Date', 'Time_Orderd',  'Time_Order_picked',  'Weather_conditions']
final_df_5_columns = ['Order_Date', 'Time_Orderd',  'Time_Order_picked',  'Weather_conditions']

final_df_5[final_df_5_columns] = df_5_new[df_5_columns].copy()

final_df_5 = pd.concat([final_df_5, df_5_new], ignore_index=True)



Shuffle final dataset

In [ ]:
# Shuffle the rows
final_df_5 = final_df_5.sample(frac=1, random_state=42)  # frac=1 means sample all rows

Dataset D2

In [6]:
df_2 = pd.read_csv("/root/.cache/kagglehub/datasets/benroshan/online-food-delivery-preferencesbangalore-region/versions/3/onlinedeliverydata.csv")

To populate the values of order preferrence to Swiggy and Zomato

In [ ]:
val = ['Zomato', 'Swiggy']
for i in df_2['Medium (P1)']:
  if str(i) == "Food delivery apps":
    df_2['Medium (P1)'] = np.random.choice(val, size=len(df_2))

Getting the necessary columns from D2.

In [ ]:
final_df_2 = pd.DataFrame()

df_2_columns = ['Age', 'Gender', 'Marital Status', 'Occupation', 'Monthly Income',  'Medium (P1)', 'Meal(P1)', 'Influence of rating', 'Freshness ', 'Temperature', 'Good Taste ', 'Good Quantity', 'Good Food quality', 'Poor Hygiene', 'Bad past experience', 'High Quality of package', 'Late Delivery', 'Long delivery time',  'Delay of delivery person getting assigned', 'Delay of delivery person picking up food', 'Maximum wait time', 'Less Delivery time', 'Wrong order delivered',  'Missing item', 'Order placed by mistake']

final_df_2_columns = ['Age', 'Gender', 'Marital_Status', 'Occupation', 'Monthly_Income',  'Order_Preference', 'Meal_Type', 'Influence_of_Rating', 'Freshness_of_Food', 'Temperature_of_Food', 'Taste_of_Food', 'Quantity_of_Food', 'Food_Quality', 'Poor_Hygiene', 'Bad_Past_Experience', 'High_Quality_of_Package', 'Late_Delivery', 'Long_Delivery_Time', 'Delay_of_Delivery_Person_Getting_Assigned', 'Delay_of_Delivery_Person_Picking_Up_Food', 'Maximum_Wait_Time', 'Less_Delivery_Time', 'Wrong_Order_Delivered',  'Missing_Item', 'Order_Placed_by_Mistake' ]

final_df_2[final_df_2_columns] = df_2[df_2_columns].copy()

Generating the Final Dataset

In [ ]:
final_df = pd.concat([d1_d3_d4,final_df_2,final_df_5], axis = 1)


Generating the ages for the final dataset

In [ ]:
num_rows = len(final_df)
mean_age = 30
std_dev = 8  # Standard deviation determines the spread

# Generate ages from a normal distribution
ages = np.random.normal(loc=mean_age, scale=std_dev, size=num_rows)

# Clip ages to ensure they fall within the range 18-59
ages = np.clip(ages, 18, 60)

# Assign to the DataFrame
final_df['Age'] = ages.astype(int)

Genrating the Nan values with the previous data which is present on the  original datasets

In [ ]:
for col in final_df.columns:
    if col not in ['Order_Id']:
      # the replace_nan_values function is on the cleaning dataset file 1,3,4
        final_df[col] = replace_nan_values(final_df,final_df[col])

Generating the final datset with the proper naming conventions of the column name.

In [ ]:
Dataset =  final_df[['Order_Id','Age','Gender','Marital_Status','Occupation','Monthly_Income', 'Order_Preference',	'Restaurant_Name'	,
        'City',	'Area',	'Cuisine',	'Veg/Non-Veg',	'Delivery_Time',	'Total_Order_Value',
      'Meal_Type',	'Dish_Liked',	'Average_Rating',	'Total_Rating_String', 	'Influence_of_Rating',
      'Freshness_of_Food', 'Temperature_of_Food',
       'Taste_of_Food', 'Quantity_of_Food', 'Food_Quality', 'Poor_Hygiene',
       'Bad_Past_Experience', 'High_Quality_of_Package', 'Late_Delivery',
       'Long_Delivery_Time', 'Delay_of_Delivery_Person_Getting_Assigned',
       'Delay_of_Delivery_Person_Picking_Up_Food', 'Maximum_Wait_Time',
       'Less_Delivery_Time', 'Wrong_Order_Delivered', 'Missing_Item',
       'Order_Placed_by_Mistake', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions']]
Dataset = Dataset.rename(columns={'Order_Preference':'App_Preference','Time_Orderd': 'Order_Time', 'Time_Order_picked': 'Order_Pickup_Time'})

In [ ]:
Dataset.to_csv('final_dataset_new.csv', index=False)